In [1]:
import pandas as pd 
import numpy as np

In [2]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib
from matplotlib import pyplot
import statsmodels
import statsmodels.api as sm
import sklearn
from sklearn.metrics import mean_squared_error
import math
from math import sqrt 

In [4]:
df = pd.read_csv("azureml://subscriptions/4f93438e-847c-469d-94ae-ef5ee130ccab/resourcegroups/project_team14/workspaces/mlnotebook/datastores/testml/paths/Final_Data/part-00000-tid-1107757838746000038-dd6987f9-5526-4378-b9fd-633c8f0e2565-1388-1-c000.csv")

In [5]:
df.head()

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
0,Papaya,Goa,North Goa,Mapusa,Papaya,Fruits,1.33,2400.0,3600.0,3000.0,2003-04-01
1,Papaya,Jammu and Kashmir,Jammu,Narwal Jammu (F&V),Other,Fruits,12.00,1150.0,1250.0,1200.0,2003-04-01
2,Papaya,NCT of Delhi,Delhi,Shahdara,Other,Fruits,0.10,150.0,300.0,250.0,2003-04-02
3,Papaya,Punjab,Bhatinda,Bathinda,Other,Fruits,4.50,400.0,450.0,425.0,2003-04-01
4,Papaya,Punjab,Ferozpur,Firozepur City,Other,Fruits,10.00,600.0,800.0,700.0,2003-04-02


In [5]:
df.shape

(51729422, 11)

In [6]:
df.dtypes

Commodity                    object
State Name                   object
District Name                object
Market Name                  object
Variety                      object
Group                        object
Arrivals (Tonnes)           float64
Min Price (Rs/Quintal)      float64
Max Price (Rs/Quintal)      float64
Modal Price (Rs/Quintal)    float64
Reported Date                object
dtype: object

In [7]:
df["Reported Date"] = pd.to_datetime(df["Reported Date"])

In [8]:
df.isnull().sum()

Commodity                   0
State Name                  0
District Name               0
Market Name                 0
Variety                     0
Group                       0
Arrivals (Tonnes)           0
Min Price (Rs/Quintal)      0
Max Price (Rs/Quintal)      0
Modal Price (Rs/Quintal)    0
Reported Date               0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51729422 entries, 0 to 51729421
Data columns (total 11 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Commodity                 object        
 1   State Name                object        
 2   District Name             object        
 3   Market Name               object        
 4   Variety                   object        
 5   Group                     object        
 6   Arrivals (Tonnes)         float64       
 7   Min Price (Rs/Quintal)    float64       
 8   Max Price (Rs/Quintal)    float64       
 9   Modal Price (Rs/Quintal)  float64       
 10  Reported Date             datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 4.2+ GB


In [9]:
df2 = df.copy()

In [10]:
df2.dtypes

Commodity                           object
State Name                          object
District Name                       object
Market Name                         object
Variety                             object
Group                               object
Arrivals (Tonnes)                  float64
Min Price (Rs/Quintal)             float64
Max Price (Rs/Quintal)             float64
Modal Price (Rs/Quintal)           float64
Reported Date               datetime64[ns]
dtype: object

In [11]:
garr = df2.groupby("Commodity").agg(avg_arrival=('Arrivals (Tonnes)', 'mean')).round(2)

tarr = garr['avg_arrival'].sum()

avgarr = garr.assign(percentage=lambda x: (x['avg_arrival'] / tarr) * 100)

ranks = avgarr.sort_values(by='percentage', ascending=False)

print(ranks)

                     avg_arrival  percentage
Commodity                                   
GladiolusCutFlower        695.04    6.146037
Jaffri                    681.43    6.025688
RoseLocal                 660.00    5.836189
Lilly                     550.00    4.863491
Marget                    542.52    4.797347
...                          ...         ...
Maragensu                   1.00    0.008843
RatTailRadishMogari         1.00    0.008843
Lukad                       0.63    0.005571
Gramflour                   0.43    0.003802
Jamamkhan                   0.24    0.002122

[297 rows x 2 columns]


In [13]:
newcols = ranks.head(30)
newcols

,avg_arrival,percentage
Commodity,,
GladiolusCutFlower,695.04,6.146037
Jaffri,681.43,6.025688
RoseLocal,660.00,5.836189
Lilly,550.00,4.863491
Marget,542.52,4.797347
TenderCoconut,342.72,3.030574
RoseLoose,269.17,2.380192
Hen,248.86,2.200597
Bamboo,237.50,2.100144


In [14]:
unique_commodities = newcols.index.unique()

# Print unique commodity names
print(unique_commodities)

Index(['GladiolusCutFlower', 'Jaffri', 'RoseLocal', 'Lilly', 'Marget',
       'TenderCoconut', 'RoseLoose', 'Hen', 'Bamboo', 'TubeRoseSingle',
       'ChrysanthemumLoose', 'BroomstickFlowerBroom', 'Carnation',
       'PaddyDhanBasmati', 'TubeFlower', 'Egg', 'Cock', 'CottonSeed',
       'SkinAndHide', 'Coconut', 'Wood', 'Cotton', 'DailaChandni',
       'MyrobolanHarad', 'TubeRoseLoose', 'PaddyDhanCommon', 'Tobacco',
       'Sugarcane', 'Soyabean', 'Raibel'],
      dtype='object', name='Commodity')


In [15]:
a = df["Commodity"].unique()
print(a)

['Papaya' 'PointedgourdParval' 'HybridCumbu' 'AmlaNelliKai' 'DryChillies'
 'Guar' 'Lint' 'SeasonLeaves' 'KulthiHorseGram'
 'KabuliChanaChickpeas_White' 'TenderCoconut' 'Thondekai'
 'Pepperungarbled' 'Pineapple' 'BengalGramGramWhole' 'Bottlegourd'
 'Bamboo' 'Drumstick' 'Seemebadnekai' 'Safflower'
 'SesamumSesame,Gingelly,Til' 'ChillyCapsicum' 'Tea' 'ChowChow'
 'CinamonDalchini' 'Peascod' 'Mustard' 'NigerSeedRamtil' 'Orange'
 'RagiFingerMillet' 'Cow' 'ElephantYamSuran' 'GroundNutOil' 'JowarSorghum'
 'GladiolusCutFlower' 'GramRawChholia' 'TamarindSeed' 'Turnip' 'Yam'
 'Peach' 'SkinAndHide' 'Beans' 'GroundnutSplit' 'Banana' 'Ox' 'Maize'
 'Carnation' 'Sarasum' 'RoseLocal' 'IsabgulPsyllium' 'ChikoosSapota'
 'TubeRoseLoose' 'Coca' 'ChrysanthemumLoose' 'Plum' 'BigGram'
 'BengalGramDalChanaDal' 'BetalLeaves' 'Borehannu' 'ArharDalTurDal'
 'BarleyJau' 'Gurellu' 'MustardOil' 'NeemSeed' 'CorianderLeaves'
 'FieldPea' 'Capsicum' 'Millets' 'GreenPeas' 'Roundgourd' 'Khoya'
 'GingerDry' 'GreenAvareW' 'T

In [16]:
cols =['GladiolusCutFlower', 'Jaffri', 'RoseLocal', 'Lilly', 'Marget',
       'TenderCoconut', 'RoseLoose', 'Hen', 'Bamboo', 'TubeRoseSingle',
       'ChrysanthemumLoose', 'BroomstickFlowerBroom', 'Carnation',
       'PaddyDhanBasmati', 'TubeFlower', 'Egg', 'Cock', 'CottonSeed',
       'SkinAndHide', 'Coconut', 'Wood', 'Cotton', 'DailaChandni',
       'MyrobolanHarad', 'TubeRoseLoose', 'PaddyDhanCommon', 'Tobacco',
       'Sugarcane', 'Soyabean', 'Raibel']
top = newcols.index.tolist()

# Create a new DataFrame with only the top 10 commodity rows
df2 = df2[df2['Commodity'].isin(top)].copy()


In [17]:
df2.head()

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
1415951,TenderCoconut,Jharkhand,Dhanbad,Dhanbad,Other,Fruits,34.0,600.0,650.0,650.0,2003-04-04
1415952,TenderCoconut,Jharkhand,Dhanbad,Dhanbad,Other,Fruits,40.0,650.0,700.0,700.0,2003-04-02
1415953,TenderCoconut,Karnataka,Hassan,Belur,Tender Coconut,Fruits,400.0,2000.0,2500.0,2300.0,2003-04-04
1415954,TenderCoconut,Karnataka,Hassan,Belur,Tender Coconut,Fruits,200.0,2000.0,2300.0,2100.0,2003-04-01
1415955,TenderCoconut,Karnataka,Chamrajnagar,Chamaraj Nagar,Tender Coconut,Fruits,500.0,2600.0,2600.0,2600.0,2003-04-03


In [18]:
df2.tail()

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
49957190,Hen,NCT of Delhi,Delhi,"Fish,Poultry & Egg Market, Gazipur",Boiler/Firm(White-Brown),"Live Stock,Poultry,Fisheries",248.86,25.0,35.0,30.0,2021-11-01
49957191,Hen,Odisha,Dhenkanal,Dhenkanal,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",520.00,16000.0,17000.0,17000.0,2021-11-03
49957192,Hen,Kerala,Kottayam,Pampady,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",250.00,12000.0,12400.0,12200.0,2022-11-03
49957193,Hen,Kerala,Kottayam,Pampady,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",248.86,13000.0,13400.0,13200.0,2022-11-02
49957194,Hen,Kerala,Kottayam,Pampady,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",248.86,11000.0,11400.0,11200.0,2022-11-01


In [19]:
df2.shape

(5086679, 11)

In [20]:
df2

,Commodity,State Name,District Name,Market Name,Variety,Group,Arrivals (Tonnes),Min Price (Rs/Quintal),Max Price (Rs/Quintal),Modal Price (Rs/Quintal),Reported Date
1415951,TenderCoconut,Jharkhand,Dhanbad,Dhanbad,Other,Fruits,34.00,600.0,650.0,650.0,2003-04-04
1415952,TenderCoconut,Jharkhand,Dhanbad,Dhanbad,Other,Fruits,40.00,650.0,700.0,700.0,2003-04-02
1415953,TenderCoconut,Karnataka,Hassan,Belur,Tender Coconut,Fruits,400.00,2000.0,2500.0,2300.0,2003-04-04
1415954,TenderCoconut,Karnataka,Hassan,Belur,Tender Coconut,Fruits,200.00,2000.0,2300.0,2100.0,2003-04-01
1415955,TenderCoconut,Karnataka,Chamrajnagar,Chamaraj Nagar,Tender Coconut,Fruits,500.00,2600.0,2600.0,2600.0,2003-04-03
...,...,...,...,...,...,...,...,...,...,...,...
49957190,Hen,NCT of Delhi,Delhi,"Fish,Poultry & Egg Market, Gazipur",Boiler/Firm(White-Brown),"Live Stock,Poultry,Fisheries",248.86,25.0,35.0,30.0,2021-11-01
49957191,Hen,Odisha,Dhenkanal,Dhenkanal,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",520.00,16000.0,17000.0,17000.0,2021-11-03
49957192,Hen,Kerala,Kottayam,Pampady,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",250.00,12000.0,12400.0,12200.0,2022-11-03
49957193,Hen,Kerala,Kottayam,Pampady,Boiler/Firm(White),"Live Stock,Poultry,Fisheries",248.86,13000.0,13400.0,13200.0,2022-11-02


In [21]:
# independent var 
x = df2[["Commodity","State Name","District Name","Market Name"]]

In [22]:
# dependent var
y = df2[["Modal Price (Rs/Quintal)"]]

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [24]:
x_train

,Commodity,State Name,District Name,Market Name
40424080,PaddyDhanCommon,Chattisgarh,Mahasamund,Bhavarpur
42191940,PaddyDhanCommon,Rajasthan,Kota,Kota
41770340,PaddyDhanCommon,Telangana,Warangal,Kesamudram
42162327,PaddyDhanCommon,Telangana,Khammam,Dammapet
41769945,PaddyDhanCommon,Telangana,Nalgonda,Chandur(Mungodu)
...,...,...,...,...
39788555,PaddyDhanCommon,Andhra Pradesh,Krishna,Pamarru
39818291,PaddyDhanCommon,Pondicherry,Pondicherry,Thattanchavady
39783928,PaddyDhanCommon,West Bengal,Bankura,Bishnupur(Bankura)
32842585,Cotton,Telangana,Khammam,Madhira


In [25]:
x_test

,Commodity,State Name,District Name,Market Name
41187030,PaddyDhanCommon,Telangana,Nalgonda,Bhongir
26843353,Soyabean,Gujarat,Sabarkantha,Modasa
40605543,PaddyDhanCommon,Telangana,Nalgonda,Devarkonda(Mallepalli)
26871808,Soyabean,Maharashtra,Vashim,Risod
42186008,PaddyDhanCommon,Uttar Pradesh,Bahraich,Naanpara
...,...,...,...,...
42898193,PaddyDhanCommon,West Bengal,Bankura,Indus
28882963,Soyabean,Maharashtra,Nandurbar,Shahada
33169044,Cotton,Telangana,Adilabad,Indravelly(Utnoor)
14487055,Coconut,Kerala,Idukki,Thodupuzha


In [26]:
y_train

,Modal Price (Rs/Quintal)
40424080,1300.0
42191940,2150.0
41770340,1199.0
42162327,1450.0
41769945,1345.0
...,...
39788555,1450.0
39818291,650.0
39783928,710.0
32842585,3700.0


In [27]:
y_test

,Modal Price (Rs/Quintal)
41187030,660.0
26843353,2200.0
40605543,1888.0
26871808,6800.0
42186008,0.0
...,...
42898193,1170.0
28882963,6281.0
33169044,5400.0
14487055,3700.0


In [31]:
print(df2['Commodity'].value_counts())

Commodity
PaddyDhanCommon          2876206
Soyabean                  990164
Cotton                    791110
Coconut                   243705
PaddyDhanBasmati           57355
TenderCoconut              42581
Wood                       38462
Tobacco                    13018
Hen                         9685
Cock                        7289
RoseLoose                   5394
CottonSeed                  2270
Egg                         2115
ChrysanthemumLoose          1383
RoseLocal                    842
DailaChandni                 722
Carnation                    671
Raibel                       611
Lilly                        556
TubeRoseLoose                536
GladiolusCutFlower           520
Jaffri                       447
Marget                       407
TubeFlower                   363
Sugarcane                    127
BroomstickFlowerBroom         59
TubeRoseSingle                42
MyrobolanHarad                34
Bamboo                         4
SkinAndHide                    1


In [32]:
from sklearn.preprocessing import OneHotEncoder

In [33]:
ohe = OneHotEncoder()

In [35]:
x_train_new = ohe.fit_transform(x_train).toarray()

In [38]:
x_train_new

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [37]:
x_test_new = ohe.fit_transform(x_test).toarray()

In [41]:
x_test_new

array([0., 0., 0., ..., 0., 0., 0.])

In [1]:
pd.get_dummies(x_test_new)

NameError: name 'pd' is not defined